#Fine-tuning UBC-NLP AraT5 models
https://github.com/UBC-NLP/araT5

##Install requirments

In [ ]:
!pip install transformers==4.5.1 nltk datasets regex sklearn sentencepiece protobuf fairscale sacrebleu rouge_score -qqq

##Get seq2seq trainier from UBC-NLP/araT5 github

In [ ]:
!wget https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/run_trainier_seq2seq_huggingface.py?token=AA4R7KKKCBPS5WX4BJSWNFTBG2OPK -O run_trainier_seq2seq_huggingface.py
!wget https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/eval_squad.py?token=AA4R7KOBBXSPCDXRRHJ3RW3BG2RF4 -O eval_squad.py

--2021-08-30 23:45:27--  https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/run_trainier_seq2seq_huggingface.py?token=AA4R7KKKCBPS5WX4BJSWNFTBG2OPK
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36633 (36K) [text/plain]
Saving to: ‘run_trainier_seq2seq_huggingface.py’

run_trainier_seq2se 100%[===================>]  35.77K  --.-KB/s    in 0.001s  

2021-08-30 23:45:28 (58.6 MB/s) - ‘run_trainier_seq2seq_huggingface.py’ saved [36633/36633]

--2021-08-30 23:45:28--  https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/eval_squad.py?token=AA4R7KOBBXSPCDXRRHJ3RW3BG2RF4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubus

##Fine-tuning AraT5 on title generation task
In this example, we use **sample data from ARGEN title genration dataset** as an example for how to **fine-tunne AraT5 models**

###Getting ARGEN title genration sample train and valid

In [ ]:
!wget https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/ARGEn_title_genration_sample_train.tsv?token=AA4R7KKFIZA3WUAVHFI7AIDBG2QCE -O ARGEn_title_genration_sample_train.tsv
!wget https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/ARGEn_title_genration_sample_valid.tsv?token=AA4R7KLXSAQEF2HUWL52Y53BG2QEE -O ARGEn_title_genration_sample_valid.tsv

--2021-08-30 23:45:30--  https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/ARGEn_title_genration_sample_train.tsv?token=AA4R7KKFIZA3WUAVHFI7AIDBG2QCE
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2961751 (2.8M) [text/plain]
Saving to: ‘ARGEn_title_genration_sample_train.tsv’

ARGEn_title_genrati 100%[===================>]   2.82M  --.-KB/s    in 0.03s   

2021-08-30 23:45:30 (93.2 MB/s) - ‘ARGEn_title_genration_sample_train.tsv’ saved [2961751/2961751]

--2021-08-30 23:45:30--  https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/ARGEn_title_genration_sample_valid.tsv?token=AA4R7KLXSAQEF2HUWL52Y53BG2QEE
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting t

###Training the model
**To obtain the best results, you should train the model for 22 epochs with full data (train, valid, and test)**

In [ ]:
!python run_trainier_seq2seq_huggingface.py \
        --learning_rate 5e-5 \
        --max_target_length 128 --max_source_length 128 \
        --per_device_train_batch_size 8 --per_device_eval_batch_size 8 \
        --model_name_or_path "UBC-NLP/AraT5-base" \
        --output_dir "/content/AraT5_FT_title_generation" --overwrite_output_dir \
        --num_train_epochs 3 \
        --train_file "/content/ARGEn_title_genration_sample_train.tsv" \
        --validation_file "/content/ARGEn_title_genration_sample_valid.tsv" \
        --task "title_generation" --text_column "document" --summary_column "title" \
        --load_best_model_at_end --metric_for_best_model "eval_bleu" --greater_is_better True --evaluation_strategy epoch --logging_strategy epoch --predict_with_generate\
        --do_train --do_eval

###Generating titles
(1) using trained checkpoint

In [ ]:
!python run_trainier_seq2seq_huggingface.py \
        --learning_rate 5e-5 \
        --max_target_length 128 --max_source_length 128 \
        --per_device_train_batch_size 8 --per_device_eval_batch_size 8 \
        --model_name_or_path "/content/AraT5_FT_title_generation" \
        --output_dir "/content/AraT5_FT_title_generation" \
        --train_file "/content/ARGEn_title_genration_sample_train.tsv" \
        --validation_file "/content/ARGEn_title_genration_sample_valid.tsv" \
        --test_file "/content/ARGEn_title_genration_sample_valid.tsv" \
        --metric_for_best_model "eval_bleu" \
        --task "title_generation" --text_column "document" --summary_column "title" \
        --predict_with_generate --do_predict

(2) using **UBC-NLP/AraT5-base-title-generation** model

In [ ]:

!python run_trainier_seq2seq_huggingface.py \
        --learning_rate 5e-5 \
        --max_target_length 128 --max_source_length 128 \
        --per_device_train_batch_size 8 --per_device_eval_batch_size 8 \
        --model_name_or_path "UBC-NLP/AraT5-base-title-generation" \
        --output_dir "/content/AraT5-base-title-generation_output" \
        --train_file "/content/ARGEn_title_genration_sample_train.tsv" \
        --validation_file "/content/ARGEn_title_genration_sample_valid.tsv" \
        --test_file "/content/ARGEn_title_genration_sample_valid.tsv" \
        --metric_for_best_model "eval_bleu" \
        --task "title_generation" --text_column "document" --summary_column "title" \
        --predict_with_generate --do_predict

last_checkpoint None
08/31/2021 00:49:07 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/31/2021 00:49:07 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/content/AraT5-base-title-generation_output', overwrite_output_dir=False, do_train=False, do_eval=None, do_predict=True, evaluation_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, logging_dir='runs/Aug31_00-49-07_ca4daf48a05c', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_

In [ ]:
#top 10 titles genrated from our full finetuned model (UBC-NLP/AraT5-base-title-generation)
!head -10 /content/AraT5-base-title-generation_output/test_preds_seq2seq.txt

وزارة النقل: تنفيذ 3 آلاف كلم من الطرق والجسور بالولايات الثلاث
البنوك السودانية تتوقع آثار إيجابية على الاقتصاد والمواطن بعد قرار البنك المركزي
الغرفة النقابية الوطنية للباعثين العقاريين: ارتفاع أسعار العقارات وتوقعات بارتفاع في الأسعار
أسعارالعملات الاجنبية مقابل الجنيه السوداني ليوم الاثنين الموافق24سبتمبر2018
الإسكان: تخصيص 300 ألف شقة لمحدودى الدخل
الذهب يهبط لأدنى مستوى في 3 أشهر ونصف
بسبب خسائر فادحة تكبدتهاهم خلال أسبوع فقط مصالح الجمارك التونسية تشرع في حملة لمحاربة التهريب على الحدود الجزائرية
الهيئة الملكية بالجبيل توقع مذكرات تفاهم مع شركات عالمية
البرلمان يوافق على تصدير إناث الثروة الحيوانية
وزير المالية: التمديد 3 سنوات في منظومة الامتيازات لن يخلق سابقة
